In [43]:
import traceback
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, Tool
from langchain_experimental.tools import PythonREPLTool



def execute_python_code(code: str) -> dict:
    """
    Executes the provided Python code and returns the result or an error message.
    """
    local_context = {}
    try:
        exec(code, {}, local_context)
        return {"result": local_context}
    except Exception as e:
        return {"error": traceback.format_exc()}

class PythonTool(BaseTool):
    name: str = "PythonExecutor"
    description: str = "Tool for executing Python code."

    def _run(self, code: str) -> str:
        """
        Runs the given Python code and returns the output or error.
        """
        result = execute_python_code(code)
        if "error" in result:
            return f"Error: {result['error']}"
        return f"Result: {result['result']}"

    async def _arun(self, code: str) -> str:
        raise NotImplementedError("This tool does not support async execution.")

# Example of registering and using the tool
tools2 = [
    Tool(
        name="PythonExecutor",
        func=PythonTool()._run,
        description="Tool for executing Python code"
    )
]

tools = [PythonREPLTool()]



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Load the 16-bit model
model_name = "llama8"  # Replace with your model's name
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Save the quantized model
model.save_pretrained("quantized_model")
tokenizer.save_pretrained("quantized_model")


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [44]:
print(llm.invoke('please give me code to create and run training of simple pytorch model, write whole code'))

APIConnectionError: Connection error.

In [46]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import httpx
# Inicjalizacja narzędzia
python_tool = PythonTool()

# Inicjalizacja LLM
llm = OpenAI(
    temperature=0,
    openai_api_base="http://localhost:8000/v1",  # Endpoint API
    model="llama3b",  # Nazwa modelu
    api_key='EMPTY'
)

# Ustawienie agenta z narzędziem
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_yterations = 50
)

# Zapytanie do agenta
response = agent.invoke("please run training of a very simple pytorch model on random data for 5 epochs, code should be really short")
print(response)




> Entering new AgentExecutor chain...
 I need to create a simple PyTorch model and train it on random data
Action: Python_REPL
Action Input: from torch import nn, torch.randn; model = nn.Linear(1, 1); model.train()
Observation: SyntaxError('invalid syntax', ('<string>', 1, 28, 'from torch import nn, torch.randn; model = nn.Linear(1, 1); model.train()\n', 1, 29))
Thought: I need to fix the syntax error
Action: Python_REPL
Action Input: from torch import nn, torch.randn; model = nn.Linear(1, 1); model.train(); print(model)
Observation: SyntaxError('invalid syntax', ('<string>', 1, 28, 'from torch import nn, torch.randn; model = nn.Linear(1, 1); model.train(); print(model)\n', 1, 29))
Thought: I need to fix the syntax error
Action: Python_REPL
Action Input: from torch import nn, torch.randn; model = nn.Linear(1, 1); model.train(); print(model)
Observation: SyntaxError('invalid syntax', ('<string>', 1, 28, 'from torch import nn, torch.randn; model = nn.Linear(1, 1); model.train(); print(

In [32]:
import traceback
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, Tool

# Function to execute the code
def execute_python_code(code: str) -> dict:
    """
    Executes the provided Python code and returns the result or an error message.
    """
    local_context = {}
    print(code)

    try:
        print('code',code)
        result = exec_with_return(code)  # Execute the Python code in a new context
        return {"result": result}  # Return the result
    except Exception as e:
        return {"error": traceback.format_exc()}  # Return the error message

# Python tool class to execute code
class PythonTool(BaseTool):
    name: str = "PythonExecutor"
    description: str = "Tool for executing Python code. "

    def _run(self, code: str) -> str:
        """
        Runs the given Python code and returns the output or error.
        """
        result = execute_python_code(code)
        if "error" in result:
            return f"Error: {result['error']}"  # Return formatted error
        return f"Result: {result['result']}"  # Return the result

    async def _arun(self, code: str) -> str:
        raise NotImplementedError("This tool does not support async execution.")

# Registering the tool with agent
tools = [
    Tool(
        name="PythonExecutor",
        func=PythonTool()._run,
        description="Tool for executing Python code"
    )
]

# Initialize the agent with the tools
agent = initialize_agent(tools,llm, agent_type="zero-shot-react-description", verbose=True)

# Input example (the problem case: Calculate 2+10)
input_text = "Calculate 2 + 10 in Python"

# Running the agent to handle the input
output = agent.run(input_text)
print(output)


/tmp/ipykernel_17646/1174991839.py:53: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = agent.run(input_text)




> Entering new AgentExecutor chain...
 I need to use the PythonExecutor to execute the Python code
Action: PythonExecutor
Action Input: "print(2 + 10)"print(2 + 10)
code print(2 + 10)

Observation: Error: Traceback (most recent call last):
  File "/tmp/ipykernel_17646/1174991839.py", line 15, in execute_python_code
    result = exec_with_return(code)  # Execute the Python code in a new context
NameError: name 'exec_with_return' is not defined

Thought: I need to define the exec_with_return function
Action: PythonExecutor
Action Input: "def exec_with_return(code):\n    return eval(code)"def exec_with_return(code):\n    return eval(code)
code def exec_with_return(code):\n    return eval(code)

Observation: Error: Traceback (most recent call last):
  File "/tmp/ipykernel_17646/1174991839.py", line 15, in execute_python_code
    result = exec_with_return(code)  # Execute the Python code in a new context
NameError: name 'exec_with_return' is not defined

Thought: I need to define the exec

In [51]:
PythonTool()._run("2 + 2")

'Result: 4'

In [52]:
exec("2 + 2")

In [39]:
program = 'a = 5\nb=10\nprint("Sum =", a+b)'
exec(program)

Sum = 15


In [47]:
import ast
import copy
def convertExpr2Expression(Expr):
        Expr.lineno = 0
        Expr.col_offset = 0
        result = ast.Expression(Expr.value, lineno=0, col_offset = 0)

        return result
def exec_with_return(code):
    code_ast = ast.parse(code)

    init_ast = copy.deepcopy(code_ast)
    init_ast.body = code_ast.body[:-1]

    last_ast = copy.deepcopy(code_ast)
    last_ast.body = code_ast.body[-1:]

    exec(compile(init_ast, "<ast>", "exec"), globals())
    if type(last_ast.body[0]) == ast.Expr:
        return eval(compile(convertExpr2Expression(last_ast.body[0]), "<ast>", "eval"),globals())
    else:
        exec(compile(last_ast, "<ast>", "exec"),globals())

result = exec_with_return("2 + 2")

In [48]:
result

4

In [33]:
# Function to execute the code
def execute_python_code(code: str) -> dict:
    local_context = {}
    try:
        result = exec(code, local_context)  # Execute the Python code in a new context
        return {"result": local_context}
    except Exception as e:
        return {"error": traceback.format_exc()}  # Return the error message

# Python tool class to execute code
class PythonTool(BaseTool):
    name: str = "PythonExecutor"
    description: str = "Tool for executing Python code. Insert python code action to run"

    def _run(self, code: str) -> str:
        """
        Runs the given Python code and returns the output or error.
        """
        result = execute_python_code(code)
        if "error" in result:
            return f"Error: {result['error']}"  # Return formatted error
        return f"Result: {result['result']}"  # Return the result

    async def _arun(self, code: str) -> str:
        raise NotImplementedError("This tool does not support async execution.")

# Registering the tool with agent
tools = [
    Tool(
        name="PythonExecutor",
        func=PythonTool()._run,
        description="Tool for executing Python code"
    )
]

# Initialize the agent with the tools
agent = initialize_agent(tools, llm, agent_type="zero-shot-react-description", verbose=True)

# Input example (the problem case: Calculate 2+10)
input_text = "Calculate 2 + 10 in Python"

# Running the agent to handle the input
output = agent.run(input_text)
print(output)




> Entering new AgentExecutor chain...
 I need to use the PythonExecutor to execute the Python code
Action: PythonExecutor
Action Input: "print(2 + 10)"12

Observation: Result: {'__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'), '__build_class__': <built-in function __build_class__>, '__import__': <built-in function __import__>, 'abs': <built-in function abs>, 'all': <built-in function all>, 'any': <built-in function any>, 'ascii': <built-in function ascii>, 'bin': <built-in function bin>, 'breakpoint': <built-in function breakpoint>, 'callable': <built-in function callable>, 'chr': <built-in function chr>, 'compile': <built-in function compile>, 'delattr': <built-in 

In [24]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [35]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI
tools = [PythonREPLTool()]
instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
agent = create_openai_functions_agent(ChatOpenAI(temperature=0,model='llama3b' ,openai_api_base='http://localhost:8000/v1', api_key='EMPTY'), tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What is the 10th fibonacci number?"})



/home/adam/miniconda3/envs/llm/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'object': 'error', 'message': "[{'type': 'extra_forbidden', 'loc': ('body', 'functions'), 'msg': 'Extra inputs are not permitted', 'input': [{'name': 'Python_REPL', 'description': 'A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.', 'parameters': {'properties': {'query': {'type': 'string'}}, 'required': ['query'], 'type': 'object'}}], 'url': 'https://errors.pydantic.dev/2.9/v/extra_forbidden'}]", 'type': 'BadRequestError', 'param': None, 'code': 400}